<a href="https://colab.research.google.com/github/mahbubcsedu/interviewcoding/blob/main/TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Understanding the Traveling Salesman Problem (TSP)

The **Traveling Salesman Problem (TSP)** is one of the most famous combinatorial optimization problems in computer science and operations research. The problem can be stated as:

> **Given a set of cities and the distances between every pair of cities, find the shortest possible route that visits every city exactly once and returns to the starting city.**

The TSP is NP-hard, meaning there is no known polynomial-time solution for the general case (for large numbers of cities). However, there are several approaches to solve TSP:

1. **Brute Force** (Exponential Time)
2. **Dynamic Programming** (Held-Karp Algorithm, \( O(n^2 \cdot 2^n) \))
3. **Greedy Algorithms** (Approximation)
4. **Approximation Algorithms** (Christofides’ algorithm for metric TSP)
5. **Backtracking and Branch-and-Bound**

### Approach to Solve TSP

1. **Brute Force**:
   - Generate all permutations of the cities.
   - Calculate the cost of visiting all cities and return to the starting city.
   - This is computationally expensive (\(O(n!)\)) but guarantees an optimal solution.

2. **Dynamic Programming (Held-Karp Algorithm)**:
   - This is a more efficient approach that solves TSP in \(O(n^2 \cdot 2^n)\) time using dynamic programming. It works by storing intermediate results to avoid redundant calculations.

   **State Representation**:
   - Define `dp[mask][i]` to be the shortest path that visits all cities in the bitmask `mask` and ends at city `i`.
   - The transition is:
     - `dp[mask][i] = min(dp[mask ^ (1 << i)][j] + dist[j][i])` where `j` is any city in the subset represented by `mask ^ (1 << i)` and `dist[j][i]` is the distance from city `j` to city `i`.

3. **Greedy Heuristic**:
   - A simple greedy approach that chooses the nearest unvisited city at each step. While it is much faster than the exact solutions, it doesn't always give the optimal result.

4. **Backtracking/Branch and Bound**:
   - This approach tries to find the solution by branching and pruning the search space based on constraints.

### Problem Example 1: Solving TSP Using Dynamic Programming

Let's take a problem where we solve TSP using the **Held-Karp algorithm** (dynamic programming approach). Here's a typical problem statement:

### Problem Statement

**Input:**
- A matrix `dist[i][j]` representing the distance between city `i` and city `j` (assumed to be symmetric).
- `n`: The number of cities.

**Output:**
- The length of the shortest possible route that visits each city exactly once and returns to the starting city.

**Example Input:**

```
dist = [
    [0, 10, 15, 20, 25],
    [10, 0, 35, 25, 30],
    [15, 35, 0, 30, 5],
    [20, 25, 30, 0, 10],
    [25, 30, 5, 10, 0]
]
```

**Example Output:**
```
80
```

This example represents 5 cities with distances between each pair. The optimal path is one that visits each city exactly once and returns to the start with the minimum total distance.

### Python Code to Solve TSP Using Dynamic Programming

```python
def tsp(dist):
    n = len(dist)
    
    # DP table: dp[mask][i] represents the minimum path cost to visit cities in 'mask' ending at city 'i'
    dp = [[float('inf')] * n for _ in range(1 << n)]
    
    # Base case: starting at city 0 with only city 0 visited
    dp[1][0] = 0
    
    # Iterate through all subsets of cities
    for mask in range(1 << n):
        for u in range(n):  # Try to end at city u
            if mask & (1 << u):  # If u is in the subset represented by mask
                for v in range(n):  # Try to transition from city v to city u
                    if u != v and (mask & (1 << v)):  # v must be visited before u
                        dp[mask][u] = min(dp[mask][u], dp[mask ^ (1 << u)][v] + dist[v][u])
    
    # The answer is the minimum cost to visit all cities and return to city 0
    return min(dp[(1 << n) - 1][i] + dist[i][0] for i in range(1, n))

# Example usage
dist = [
    [0, 10, 15, 20, 25],
    [10, 0, 35, 25, 30],
    [15, 35, 0, 30, 5],
    [20, 25, 30, 0, 10],
    [25, 30, 5, 10, 0]
]

print(f"Minimum cost to complete TSP: {tsp(dist)}")
```

### Explanation of the Code:
1. **dp[mask][i]**: The `dp` table stores the minimum cost of visiting all cities in `mask` (where `mask` is a bitmask representing a subset of cities) and ending at city `i`.
2. **Base case**: The path starts at city 0, so `dp[1][0] = 0`.
3. **Transition**: For each subset `mask`, we try to reach city `u` from all other cities `v` already in the subset. We update `dp[mask][u]` with the minimum cost.
4. **Final result**: The minimum cost to visit all cities and return to city 0 is found by checking all possible cities `i` and adding the distance from `i` back to city 0.

### Complexity Analysis:
- **Time Complexity**: \( O(n^2 \cdot 2^n) \), where `n` is the number of cities. The `2^n` term comes from iterating over all subsets of cities, and `n^2` is from considering transitions between pairs of cities.
- **Space Complexity**: \( O(n \cdot 2^n) \), since we store a DP table of size \( 2^n \times n \).

### Problem Example 2: Approximation of TSP (Greedy Algorithm)

Another approach is to use a **greedy heuristic**, which approximates the solution but does not always guarantee the optimal result. Here's how we could solve a similar TSP problem using a simple greedy approach:

### Problem Statement

You are given a set of cities and distances between each pair of cities. Find a path that visits all cities exactly once with the minimum total cost. Use a greedy approach that at each step selects the nearest unvisited city.

### Python Code (Greedy Heuristic):

```python
def greedy_tsp(dist):
    n = len(dist)
    visited = [False] * n
    path = [0]  # Start from city 0
    visited[0] = True
    total_cost = 0
    
    for _ in range(n - 1):
        last_visited = path[-1]
        nearest_city = -1
        min_distance = float('inf')
        
        # Find the nearest unvisited city
        for i in range(n):
            if not visited[i] and dist[last_visited][i] < min_distance:
                nearest_city = i
                min_distance = dist[last_visited][i]
        
        # Add the nearest city to the path
        path.append(nearest_city)
        visited[nearest_city] = True
        total_cost += min_distance
    
    # Add the distance from the last city back to the start city
    total_cost += dist[path[-1]][0]
    
    return total_cost, path

# Example usage
dist = [
    [0, 10, 15, 20, 25],
    [10, 0, 35, 25, 30],
    [15, 35, 0, 30, 5],
    [20, 25, 30, 0, 10],
    [25, 30, 5, 10, 0]
]

cost, path = greedy_tsp(dist)
print(f"Approximate cost using greedy approach: {cost}")
print(f"Path: {path}")
```

### Explanation:
1. **Greedy Heuristic**: The algorithm starts at city 0, then at each step it chooses the nearest unvisited city, adds it to the path, and repeats until all cities are visited.
2. **Final Step**: After visiting all cities, the algorithm adds the cost of returning to the starting city (city 0).

### Complexity:
- **Time Complexity**: \( O(n^2) \), because for each city, we check all other cities to find the nearest unvisited city.
- **Space Complexity**: \( O(n) \), for storing the `visited` array and `path`.

### Conclusion

- The **Held-Karp algorithm** (Dynamic Programming) is an exact solution for TSP and works well for small to moderate-sized problems (typically up to 20-25 cities).
- The **greedy approach** is much faster,

Yes, the **Find the Shortest Superstring** problem can indeed be solved using an approach related to **Eulerian Path/Circuit** and **Hierholzer's algorithm**. Let's break it down and walk through how you would approach this problem step by step.

### Problem Overview

Given a list of strings, the goal is to find the shortest string that contains every string in the list as a substring.

### Key Insights

The problem is essentially a **shortest superstring problem**, which is a variation of the **string overlap problem**. This can be transformed into a **graph problem** where:
1. Each string is a **node** in the graph.
2. The **edges** between nodes represent the overlap between two strings.

The idea is to:
- **Maximize the overlap** between any two strings when concatenating them. This ensures that the superstring is as short as possible.
- Use **Hierholzer’s algorithm** to construct the Eulerian path that will visit each string (node) while minimizing the total length of the resulting superstring.

### Steps to Solve

1. **Construct the Overlap Graph**:
   - **Nodes** represent the strings.
   - **Edges** represent the overlap between two strings. Specifically, for each pair of strings \( S_i \) and \( S_j \), the weight of the edge is the length of the maximum suffix of \( S_i \) that overlaps with the prefix of \( S_j \).

2. **Build a Directed Graph**:
   - Construct a directed graph where each edge represents a possible concatenation between two strings with maximum overlap.
   - The weight of the edge will be the amount of overlap (e.g., if string \( S_1 \) ends with "abc" and \( S_2 \) starts with "abc", the edge weight will be 3).

3. **Eulerian Path**:
   - The problem boils down to finding a path that visits each string exactly once and minimizes the total length of the superstring.
   - Hierholzer's algorithm helps in finding such a path by determining the **Eulerian path** in this graph of strings. Each step in the path corresponds to adding a string to the superstring.

4. **Construct the Shortest Superstring**:
   - Once the Eulerian path is found, concatenate the strings in the correct order, minimizing overlap at each step.
   - If the graph is constructed correctly and the overlaps are calculated properly, the result will be the shortest superstring.

### Algorithmic Steps

1. **Calculate Overlaps**:
   For each pair of strings \( S_i \) and \( S_j \), compute the maximum overlap. This can be done using dynamic programming or by trying different prefixes and suffixes.

2. **Greedy Approach for Building the Graph**:
   Create a graph where the vertices are the strings, and the edges represent the maximal overlap between two strings. We can use dynamic programming or a greedy approach to determine the best overlap.

3. **Eulerian Path**:
   Use Hierholzer’s algorithm to construct the Eulerian path. In this case, you need to find a path where you start at a vertex and follow the edges that represent concatenations of the strings in such a way that all strings are used exactly once.

4. **Rebuild the Superstring**:
   Once the Eulerian path is found, you can reconstruct the shortest superstring by concatenating the strings, accounting for the overlaps.

### Example

Let’s look at an example with a few strings:

**Example strings**:
```
["alex", "loves", "leetcode"]
```

**Step 1: Calculate Overlaps**:
- Overlap between "alex" and "loves" is "l" (1 character).
- Overlap between "loves" and "leetcode" is "es" (2 characters).
- Overlap between "alex" and "leetcode" is "" (no overlap).

**Step 2: Build a Graph**:
- From the overlaps, create a directed graph.
  - "alex" -> "loves" with weight 1.
  - "loves" -> "leetcode" with weight 2.

**Step 3: Find Eulerian Path**:
Using **Hierholzer's algorithm**, we find a path through the graph that visits each edge exactly once.

**Step 4: Rebuild the Superstring**:
Start with "alex", then use the overlap to append "loves" ("l"), then use the overlap to append "leetcode" ("es").

**Result**:
The shortest superstring will be: `"alexlovesleetcode"`

### Python Code for Finding the Shortest Superstring

Here’s a Python implementation for the **Shortest Superstring** problem using the concepts mentioned above.

```python
def shortestSuperstring(words):
    # Helper function to calculate the maximum overlap between two strings
    def overlap(A, B):
        max_overlap = 0
        for i in range(1, min(len(A), len(B)) + 1):
            if A[-i:] == B[:i]:
                max_overlap = i
        return max_overlap
    
    n = len(words)
    # Build the overlap matrix where overlap[i][j] is the maximum overlap between words[i] and words[j]
    overlap_matrix = [[0] * n for _ in range(n)]
    
    for i in range(n):
        for j in range(n):
            if i != j:
                overlap_matrix[i][j] = overlap(words[i], words[j])
    
    # Use dynamic programming to find the shortest superstring
    dp = [[float('inf')] * n for _ in range(1 << n)]  # DP state to store the minimum superstring length
    parent = [[-1] * n for _ in range(1 << n)]  # To reconstruct the path
    
    # Initialize the DP table for single word superstrings
    for i in range(n):
        dp[1 << i][i] = len(words[i])
    
    # Iterate through all subsets of words and find the shortest superstring using overlap information
    for mask in range(1 << n):
        for u in range(n):
            if (mask & (1 << u)) == 0:
                continue
            for v in range(n):
                if mask & (1 << v):
                    continue
                new_mask = mask | (1 << v)
                new_len = dp[mask][u] + len(words[v]) - overlap_matrix[u][v]
                if new_len < dp[new_mask][v]:
                    dp[new_mask][v] = new_len
                    parent[new_mask][v] = u
    
    # Reconstruct the path
    last = min(range(n), key=lambda i: dp[(1 << n) - 1][i])
    mask = (1 << n) - 1
    order = []
    while last != -1:
        order.append(last)
        next_last = parent[mask][last]
        mask ^= (1 << last)
        last = next_last
    
    order.reverse()
    
    # Reconstruct the superstring
    superstring = words[order[0]]
    for i in range(1, len(order)):
        overlap_len = overlap(words[order[i-1]], words[order[i]])
        superstring += words[order[i]][overlap_len:]
    
    return superstring
```

### Explanation of the Code:
1. **`overlap` function**: This calculates the maximum overlap between two strings, which is used to form the directed graph of overlaps.
2. **Dynamic Programming (DP)**: The DP table `dp[mask][i]` represents the shortest superstring that covers the subset of words represented by `mask`, and ends with the `i`-th word.
3. **Reconstruction**: After filling the DP table, we reconstruct the order of words in the superstring using the `parent` table, which tracks the optimal path taken.

### Time Complexity:
- **Overlap Calculation**: The overlap between each pair of strings can be calculated in O(m * n) time, where m and n are the lengths of the strings. This gives an overall time complexity for calculating overlaps as \( O(n^2 \cdot \text{len}(word)) \).
- **DP Table Calculation**: The dynamic programming table requires checking all subsets of words (which is \( O(2^n) \)) and for each subset, checking each pair of words (which is \( O(n^2) \)), resulting in a time complexity of \( O(n^2 \cdot 2^n) \).

### Conclusion

This approach is closely related to **Hierholzer’s Algorithm** as it involves finding a path through the overlap graph (maximizing overlap between strings). The solution involves a **dynamic programming** approach combined with overlap computation, and Hierholzer's algorithm provides insight into how to optimally visit each "node" (string) in the shortest path sequence.